In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import yfinance as yf
import seaborn as sns

tickers = ["AAPL", "MSFT", "TSLA", "NVDA", "AMZN"]
benchmark = "^GSPC"  # S&P 500 for beta calculation

data = yf.download(tickers,start="2024-01-01",end="2024-12-31")["Close"]
benchmark_data = yf.download(benchmark,start="2024-01-01",end="2024-12-31")["Close"]

C:\Users\onkar\AppData\Local\Temp\ipykernel_24784\3869833015.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers,start="2024-01-01",end="2024-12-31")["Close"]
[*********************100%***********************]  5 of 5 completed
C:\Users\onkar\AppData\Local\Temp\ipykernel_24784\3869833015.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  benchmark_data = yf.download(benchmark,start="2024-01-01",end="2024-12-31")["Close"]
[*********************100%***********************]  1 of 1 completed


In [30]:
returns = data.pct_change().dropna()
benchmark_returns = benchmark_data.pct_change().dropna()
returns.head()

Ticker,AAPL,AMZN,MSFT,NVDA,TSLA
Date,,,,,
2024-01-03,-0.007488,-0.009738,-0.000728,-0.012436,-0.040134
2024-01-04,-0.012700,-0.026268,-0.007178,0.009019,-0.002181
2024-01-05,-0.004013,0.004634,-0.000516,0.022897,-0.001849
2024-01-08,0.024175,0.026577,0.018871,0.064281,0.012464
2024-01-09,-0.002263,0.015225,0.002936,0.016975,-0.022832


In [35]:
betas = {}
volatilities = {}

for ticker in tickers:
    commen_date= returns[ticker].index.intersection(benchmark_returns.index)
    stock_aligned = returns[ticker].loc[commen_date]
    benchmark_aligned = returns[ticker].loc[commen_date]

    cov = stock_aligned.cov(benchmark_aligned)
    var = benchmark_aligned.var()
    betas[ticker] = cov/var
    volatilities[ticker] = stock_aligned.std()




In [37]:
cov_matrix = returns.cov()


In [40]:
n_portfolios = 5000
mean_returns = returns.mean()

data_list = []

for _ in range(n_portfolios):
    weights = np.random.random(len(tickers))
    weights /= np.sum(weights)
    
    port_return = np.dot(weights, mean_returns)
    port_vol = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    
    avg_beta = np.dot(weights, list(betas.values()))
    avg_vol = np.dot(weights, list(volatilities.values()))
    
    data_list.append(list(weights) + [avg_beta, avg_vol, port_return, port_vol])

columns = tickers + ["Avg_Beta", "Avg_Vol", "Port_Return", "Port_Vol"]
portfolio_df = pd.DataFrame(data_list, columns=columns)


In [43]:
x = portfolio_df[tickers+["Avg_Beta","Avg_Vol"]]
y_return = portfolio_df['Port_Return']
y_vol = portfolio_df['Port_Vol']

In [48]:
from sklearn.linear_model import LinearRegression
model_return = LinearRegression(n_jobs=-1).fit(x,y_return)
model_risk = LinearRegression(n_jobs=-1).fit(x,y_vol)

In [49]:
pickle.dump(model_return, open("model_return.pkl", "wb"))
pickle.dump(model_risk, open("model_risk.pkl", "wb"))
